In [3]:
using LinearAlgebra, CodecZlib, DelimitedFiles

#### Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package.

In [7]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: creating database file
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/download.jl:63
┌ Info: reading index files
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/download.jl:81
┌ Info: dowloading index file https://math.nist.gov/MatrixMarket/matrices.html
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/downloadmm.jl:83
┌ Warning: exception during initialization: 'ProcessFailedException(Base.Process[Process(`sh -c 'curl "https://math.nist.gov/MatrixMarket/matrices.html" -Lso "/Users/tinaarputhasamy/.julia/scratchspaces/b51810bb-c9f3-55da-ae3c-350fc1fbce05/data/mm_matrices.html"'`, ProcessExited(60))])'
└ @ MatrixDepot /Users/tinaarputhasamy/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:147


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [8]:
# connectivity matrix
A = md.A

916428×916428 SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:

- How much memory does A take? If converted to a Matrix{Float64} (don't do it!), how much memory will it take?
- number of web pages
- number of edges (web links).
- number of dangling nodes (pages with no out links)
- histogram of in-degrees
- list the top 20 pages with the largest in-degrees?
- histogram of out-degrees
- which the top 20 pages with the largest out-degrees?
- visualize the sparsity pattern of  𝐀  or a submatrix of  𝐀  say A[1:10000, 1:10000].

Hint: For plots, you can use the `UnicodePlots.jl` package (spy, histogram, etc), which is fast for large data.

#### Solution:

##### - Number of Webpages

The number of webpages is equal to either the number of rows or columns in the matrix (which are equal. `This is equal to: 916,428.`

In [4]:
# code to get dimensions 

size(A, 1)

916428

##### - Number of Edges

The number of edges (web links) is equal to the number of 1's in the matrix. `This is equal to: 5,105,039.`

In [22]:
# code to get number of 1's in A

count(i ->(i==1), A)

5105039

##### - Number of Dangling Nodes 

The number of dangling nodes (pages with no out links) is equivalent to the number of rows that are completely filled with zeros. This can be obtained by finding the number of row sums that equal 0. `This is equal to: 176,974.`

In [28]:
# code to get number of dangling nodes

rowsums = sum(A, dims = 2)
count(i ->(i==0), rowsums)

# sum(A::AbstractArray; dims)
# Sum elements of an array over the given dimensions
# set dims = 1 for column sums, set dims = 2 for row sums

176974

##### - Histogram of In-degrees

The number of in-degrees per page is equal to the column sums. 

In [55]:
using UnicodePlots

colsums = sum(A, dims = 1)
histogram(colsums)

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤████████████████████████████████  916114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

##### - Top 20 pages with Largest In-degrees

In [52]:
colsums_ordered = sortperm(vec(colsums), rev= true) 
# returns indicies of sorted vector (rev = T because we want largest to smallest)
colsums_ordered[1:20] # extract top 20

20-element Vector{Int64}:
 537040
 597622
 504141
 751385
  32164
 885606
 163076
 819224
 605857
 828964
 551830
  41910
 558792
 459075
 407611
 213433
 765335
 384667
 173977
 687326

##### - Histogram of Out-degrees

The number of out-degrees per page is equal to the row sums. 

In [54]:
using UnicodePlots

rowsums = sum(A, dims = 2)
histogram(rowsums)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤████████████████████████████████  891798  
   [ 20.0,  40.0) ┤▊ 22628                                   
   [ 40.0,  60.0) ┤▏ 1329                                    
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

##### - Top 20 pages with Largest Out-degrees

In [58]:
rowsums = sum(A, dims = 2)
rowsums_ordered = sortperm(vec(rowsums), rev = true)
rowsums_ordered[1:20]

20-element Vector{Int64}:
 506743
 203749
 305230
 768092
 808644
 412411
 600480
 376429
 156951
 885729
 667585
 685696
 282141
 598189
 579315
 411594
 321092
 838279
 302734
 915274

##### - Visualize the sparsity pattern of 𝐀 or a submatrix of 𝐀 say A[1:10000, 1:10000]

In [60]:
spy(A) # entire matrix

          ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
          ┌──────────────────────────────────────────┐    
        1 │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ > 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ < 0
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│    
          │⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│  

In [61]:
spy(A[1:10000, 1:10000]) # submatrix A

         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀Sparsity Pattern⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    
         ┌──────────────────────────────────────────┐    
       1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
         │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
         │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
         │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
         │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
         │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
         │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
         │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
         │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
         │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
         │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
         │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
         │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
         │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
         │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
         │⠈⠀⠀⠄

### Q5 (75 pts) Iterative solvers

In [9]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    od        :: Vector{T} # nx1 storage for row sums (outdegrees)
    storage_d :: Vector{T} # nx1 storage for di's
    storage_a :: Vector{T} # nx1 storage for (1-p)/n 
    val1      :: Vector{T} # nx1 storage for intermediate calc 
    val2      :: Vector{T} # nx1 storage for intermediate calc 
    val3      :: Vector{T} # nx1 storage for intermediate calc 
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    #initialize
    n = size(A, 1) 
    od = A*ones(n) # calculating row sums (outdegrees) 
    # sum(A, dims = 2) doesn't seem to work because output is of type matrix...
    storage_a = zeros(n) # initializing with zero vector
    storage_d = similar(storage_a) 
    val1 = similar(storage_a)
    val2 = similar(storage_a)
    val3 = similar(storage_a)
    
    #make P
    for i in 1:n
        if od[i] == 0
            storage_d[i] = 0 
            storage_a[i] = 1/n
        else
            storage_d[i] = telep/od[i] 
            storage_a[i] = (1 - telep)/n
        end
    end

    PageRankImPt(A, telep, od, storage_d, storage_a, val1, val2, val3)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
Base.getindex(M::PageRankImPt, i, j) = (i == j) - M.rinv[j] * M.A[j, i] - M.z[j]

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    
    # P' = I - A'diag(d) - 1na' 
    # (I-P')v = v - A'diag(d)v - 1na'v 
    
    # term 1 (A'diag(d)v)
    M.val1 .= M.storage_d .* v # diag(d) * v using elementwise mult
    mul!(M.val2, transpose(M.A), M.val1) # A'*(diag(d) * v)
    
    # term 2 (1na'v)
    scalar = dot(M.storage_a, v) 
    # dot product gives scalar, don't need to preallocate in constructor 
    
    # final product ((I-P')v)
    out .= v .- M.val2 .- scalar
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    
    # P = I - diag(d)A - n1n'
    # (I-P)v = v - diag(d)Av - n1n'v
    
    # term 1 (diag(d)Av)
    mul!(M.val1, M.A, v) # getting A*v first bc want to do elementwise mult with diag(d)
    M.val2 .= M.storage_d .* M.val1 # diag(d) * A * v (using elementwise mult)  
    
    # term 2 (n1n'v)
    M.val3 .= M.storage_a .* sum(v)
    
    # final product ((I-P)v)
    out .= v .- M.val2 .- M.val3
end

In [4]:
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

In [10]:
M = PageRankImPt(A, 0.85) # note! this is not the same M = I - P^T
n = size(M, 1)

916428

In [11]:
@assert norm(transpose(M) * ones(n)) < 1e-13

In [12]:
@assert norm(M * xsol) < 1e-13